In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.metrics import log_loss


class NumpyLogisticRegression:
    
    def __init__(self, learning_rate=0.1, max_iter=100, tol=1e-3,
                 batch_size=20):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.tol = tol
        self.batch_size = batch_size
    
    def _sigmoid(self, X):
        return 1 / (1 + np.exp(-X))
    
    def _decision_function(self, X):
        return np.dot(X, self.coefs_)
    
    def decision_function(self, X):
        X = self._add_intercept(X)
        return self._decision_function(X).ravel()
    
    def _grad_nll(self, X, y):
        grad = (self._predict_proba(X) - y)
        return np.dot(X.T, grad)
    
    def _add_intercept(self, X):
        return np.hstack((X, np.ones(shape=(X.shape[0], 1))))

    def fit(self, X, y):
        X = self._add_intercept(X)
        # Make y to be a column vector for later operation
        y = np.atleast_2d(y).T
        # Initialize randomly the weights
        self.coefs_ = np.random.rand(X.shape[1], 1)
        
        it = 0
        loss = np.inf
        while it < self.max_iter and loss > self.tol:
            # select a minibatch
            idx = np.random.choice(np.arange(X.shape[0]),
                                   size=self.batch_size)
            X_subset, y_subset = X[idx], y[idx]
            # compute the gradient
            dnll = self._grad_nll(X_subset, y_subset)
            # update the parameter
            self.coefs_ -= (self.learning_rate / X_subset.shape[0]) * dnll
            # update the loss and the number of iteration
            loss = log_loss(y, self._predict_proba(X))
            it += 1
        return self
    
    def _predict_proba(self, X):
        return self._sigmoid(self._decision_function(X))
    
    def predict_proba(self, X):
        X = self._add_intercept(X)
        return self._predict_proba(X)

    def predict(self, X):
        prob = self.predict_proba(X)
        prob[prob < 0.5] = 0
        prob[prob >= 0.5] = 1
        return prob.astype(int).ravel()
    
    def score(self, X, y):
        return np.mean(y == self.predict(X))

In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
X, y = iris.data, iris.target
X = X[:, :2]
mask_class_0_1 = np.bitwise_or(y == 0, y == 1)
X = X[mask_class_0_1]
y = y[mask_class_0_1]

In [ ]:
clf = NumpyLogisticRegression(learning_rate=0.1)
clf.fit(X, y).score(X, y)

In [ ]:
def plot_2d_separator(classifier, X, ax=None, levels=None, eps=None):
    eps = X.std() / 2. if eps is None else eps
    levels = [0] if levels is None else levels
    ax = plt.gca() if ax is None else ax
    
    x_min, x_max = X[:, 0].min() - eps, X[:, 0].max() + eps
    y_min, y_max = X[:, 1].min() - eps, X[:, 1].max() + eps
    xx = np.linspace(x_min, x_max, 100)
    yy = np.linspace(y_min, y_max, 100)

    X1, X2 = np.meshgrid(xx, yy)
    X_grid = np.c_[X1.ravel(), X2.ravel()]
    decision_values = classifier.decision_function(X_grid)
    CS = ax.contour(X1, X2, decision_values.reshape(X1.shape), levels=levels)
    ax.clabel(CS, inline=1, fontsize=10)

In [ ]:
plt.scatter(X[:, 0], X[:, 1], c=y)
plt.xlabel(iris.feature_names[0])
plt.ylabel(iris.feature_names[1]);
plot_2d_separator(clf, X, levels=np.arange(-5, 5, 1))

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

In [ ]:
pipe = make_pipeline(StandardScaler(), SGDClassifier(loss='log', tol=1e-3))
cross_val_score(pipe, X, y, cv=5)

In [ ]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation
from keras import optimizers
from sklearn.metrics import accuracy_score

model = Sequential()
model.add(Dense(1, input_shape=(X.shape[1],)))
model.add(Activation("sigmoid"))
model.compile(optimizer=optimizers.SGD(lr=0.1),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.fit(X, y, epochs=10, batch_size=20)
print(f'The mean accuracy is: {accuracy_score(y, model.predict_classes(X))}')